In [1]:
# magic! (don't worry about this)
%load_ext autoreload
%autoreload 2

In [22]:
# let us import some useful things
from lib import *
from classifiers import *
import math
import numpy as np

In [93]:
# load the data same as before
microarray_file_name = '../data/leukemia_ALL_AML_matrix.txt'
labels_file_name = '../data/leukemia_ALL_AML_labels.txt'
data_store = DataSet(microarray_file_name, labels_file_name)

# we will use all our samples!
full_data_set = np.array(data_store.get_train_set() + data_store.get_test_set())

In [94]:
############################
# n-Fold Cross-Validation  #
############################

# let's partition the data into n splits 
from sklearn import cross_validation

# feel free to adjust this parameter
n = 3
max_knn_k = (n-1)*len(full_data_set)/n

# n_folds provides the indices for n partitions
# From docs: Split dataset into n consecutive folds. Each fold is then used as a validation set 
# once while the n - 1 remaining folds form the training set.
n_folds = cross_validation.KFold(len(full_data_set), n, shuffle=True)

# TODO: play around with the n_folds object to figure out what it really stores!

In [95]:
############################
# Evaluation Metrics       #
############################

# this is our function to evaluate the accuracy of the classifier
def get_accuracy(classified_samples):
    correct = [1 if guess == sample.get_label() else 0 for sample, guess in classified_samples]
    total = len(correct)
    acc = (correct.count(1) * 100.0) / total
    return acc

In [99]:
#####################
# Feature Selection #
####################

from scipy.stats import ttest_ind
from scipy.special import stdtr

# enable the t-test gene selection for binary datasets
ENABLE_SELECTION = True

samples_by_label = {}
if ENABLE_SELECTION:
    for sample in full_data_set:
        if(sample.get_label() not in samples_by_label):
            samples_by_label[sample.get_label()] = []
        samples_by_label[sample.get_label()].append(sample.get_gene_profile())
        
    # let's select our most useful genes based on the Welch's t-test!
    init_number_of_genes = len(full_data_set[0].get_gene_profile())
    selected_genes = []
    for gene_id in range(init_number_of_genes):
        profiles_by_label = [] 
        for label in samples_by_label:
            profiles_by_label.append([profile[gene_id] for profile in samples_by_label[label]])

        # we assume there are only two labels here
        t, p_value = ttest_ind(profiles_by_label[0], profiles_by_label[1], equal_var=False)
        
        if(p_value < 0.05):
            selected_genes.append(gene_id)

    print 'Number of informative genes: ' , len(selected_genes)

    # now let's update our dataset to ignore the genes that are not informative
    filtered_samples = []
    for sample in full_data_set:
        new_sample = Sample(sample.get_label(), [sample.get_gene_profile()[i] for i in range(init_number_of_genes) if i in selected_genes])
        filtered_samples.append(new_sample)

    full_data_set = np.array(filtered_samples)


Number of informative genes:  1713


In [97]:
##################
# kNN Evaluation #
##################

# feel free to change these bounds!
k_start = 1
k_end = 21

if k_end >= max_knn_k:
    print 'WARNING: your kNN bound on k exceeds the number of samples in your training set!'

knn = KNearestNeighbors()

# we are going to average the accuracy we get for each k across all the folds!
avg_acc_for_k = {}
for k in range(k_start, k_end):   
    avg_acc_for_k[k] = 0
    for train_partition, test_partition in n_folds:
        # TODO: can you figure out how the length of the train_set and test_set are affected by n?
        train_set = full_data_set[train_partition]
        test_set = full_data_set[test_partition]
    
        knn.train(train_set) 
        classified_samples = knn.classify(test_set, k, euclidean_distance)
        acc = get_accuracy(classified_samples)
        avg_acc_for_k[k] += acc
    
    avg_acc_for_k[k] /= len(n_folds)
    print 'k = ', k, ' Average accuracy: %.2f%%' % avg_acc_for_k[k]


k =  1  Average accuracy: 88.89%
k =  2  Average accuracy: 93.06%
k =  3  Average accuracy: 94.44%
k =  4  Average accuracy: 95.83%
k =  5  Average accuracy: 97.22%
k =  6  Average accuracy: 97.22%
k =  7  Average accuracy: 97.22%
k =  8  Average accuracy: 97.22%
k =  9  Average accuracy: 97.22%
k =  10  Average accuracy: 95.83%
k =  11  Average accuracy: 95.83%
k =  12  Average accuracy: 94.44%
k =  13  Average accuracy: 94.44%
k =  14  Average accuracy: 91.67%
k =  15  Average accuracy: 91.67%
k =  16  Average accuracy: 87.50%
k =  17  Average accuracy: 87.50%


KeyboardInterrupt: 

In [100]:
############################
# Decision Tree Evaluation #
############################

from sklearn import tree

# This train function is *almost* the same as the train method that 
# your knn classifier. The difference is that this is a function that
# takes as one of the parameters the decision tree, so it will be called
# like: train(decision_tree, train_samples)
def train(decision_tree, train_samples):
    feature_array = [sample.get_gene_profile() for sample in train_samples]
    labels = [sample.get_label() for sample in train_samples]
    decision_tree.fit(feature_array, labels)

def classify(decision_tree, test_samples):
    labelled_samples = []
    feature_array = [sample.get_gene_profile() for sample in test_samples]
    results = decision_tree.predict(feature_array)
    labelled_samples = [(test_samples[i], results[i]) for i in range(len(test_samples))]
    return labelled_samples

# Here, you can set criterion = "entropy" or "gini", which will determine
# what equation the decision tree will use to measure the quality of a split
#
# You can set max_features=None, "sqrt", or "log2", which will determine how many
# features the decision tree will use. Setting it to None will use all the features,
# sqrt will use sqrt(number of features) and log2 will use log2(number of features)
#
# Play around! What settings are best? Do they change for the different data sets?

n_rounds = 50

avg_acc = 0
for i in range(n_rounds):
    for train_partition, test_partition in n_folds:
        train_set = full_data_set[train_partition]
        test_set = full_data_set[test_partition]

        decision_tree = tree.DecisionTreeClassifier(criterion="gini", max_features=None)
        train(decision_tree, train_set)
        classified_samples = classify(decision_tree, test_set)
        # let's evaluate how well the classifier worked
        acc = get_accuracy(classified_samples)
        avg_acc += acc
    
avg_acc /= n_rounds*len(n_folds)
print ' Average accuracy: %.2f%%' % avg_acc

 Average accuracy: 87.28%
